In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re

df = pd.read_parquet("archive\csv\play_by_play.parquet")

df.head()

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
0,29600012,0,12,0,1,14:43 PM,12:00,None,Start of 1st Period (14:43 PM EST),None,...,None,None,0.0,0,None,NaN,None,None,None,0
1,29600012,2,10,0,1,14:50 PM,12:00,Jump Ball O'Neal vs. Kleine: Tip to Cassell,None,None,...,Suns,PHX,5.0,208,Sam Cassell,1.610613e+09,Phoenix,Suns,PHX,0
2,29600012,3,2,1,1,14:51 PM,11:45,None,None,MISS Cassell 15' Jump Shot,...,None,None,0.0,0,None,NaN,None,None,None,0
3,29600012,4,4,0,1,14:51 PM,11:43,O'Neal REBOUND (Off:0 Def:1),None,None,...,None,None,0.0,0,None,NaN,None,None,None,0
4,29600012,5,2,1,1,14:51 PM,11:29,MISS Ceballos 26' 3PT Jump Shot,None,None,...,None,None,0.0,0,None,NaN,None,None,None,0


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# Code to separate the offensive rebounds from the defensive ones and give them a new number in the column "eventmsgtype" (n 14)
df["player"] = df["homedescription"].str.extract(r"^(.*?) REBOUND")
df["offensive_rebounds"] = (
    df["homedescription"]
    .str.extract(r'Off:(\d+)')[0]
    .fillna(0)
    .astype(int)
)

df["is_offensive_rebound"] = (
    df.groupby("player")["offensive_rebounds"]
    .diff()
    .fillna(0) > 0
)

first_rebounds = df.groupby("player").head(1)
df.loc[first_rebounds.index, "is_offensive_rebound"] = (
    first_rebounds["offensive_rebounds"] > 0
)

df.loc[df["is_offensive_rebound"], "eventmsgtype"] = 14

df = df.drop(columns=["player", "offensive_rebounds", "is_offensive_rebound"])

# Same for visitordescription:
df["player"] = df["visitordescription"].str.extract(r"^(.*?) REBOUND")
df["offensive_rebounds"] = (
    df["visitordescription"]
    .str.extract(r'Off:(\d+)')[0]
    .fillna(0)
    .astype(int)
)

df["is_offensive_rebound"] = (
    df.groupby("player")["offensive_rebounds"]
    .diff()
    .fillna(0) > 0
)

first_rebounds = df.groupby("player").head(1)
df.loc[first_rebounds.index, "is_offensive_rebound"] = (
    first_rebounds["offensive_rebounds"] > 0
)

df.loc[df["is_offensive_rebound"], "eventmsgtype"] = 14

df = df.drop(columns=["player", "offensive_rebounds", "is_offensive_rebound"])

print(df)


           game_id  eventnum  eventmsgtype  eventmsgactiontype  period  \
0         29600012         0            12                   0       1   
1         29600012         2            10                   0       1   
2         29600012         3             2                   1       1   
3         29600012         4             4                   0       1   
4         29600012         5             2                   1       1   
...            ...       ...           ...                 ...     ...   
13592894  32200001       638             1                  79       4   
13592895  32200001       639             2                   1       4   
13592896  32200001       640            14                   0       4   
13592897  32200001       641             1                  79       4   
13592898  32200001       642            13                   0       4   

         wctimestring pctimestring  \
0            14:43 PM        12:00   
1            14:50 PM        12:00 

In [4]:
pd.set_option("display.max_rows", None)
df1 = df.loc[df['game_id'] == 29600012]
df1.head()

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,score,scoremargin,person1type,player1_id,player1_name,player1_team_id,player1_team_city,player1_team_nickname,player1_team_abbreviation,person2type,player2_id,player2_name,player2_team_id,player2_team_city,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
0,29600012,0,12,0,1,14:43 PM,12:00,None,Start of 1st Period (14:43 PM EST),None,None,None,0.0,0,None,NaN,None,None,None,0.0,0,None,NaN,None,None,None,0.0,0,None,NaN,None,None,None,0
1,29600012,2,10,0,1,14:50 PM,12:00,Jump Ball O'Neal vs. Kleine: Tip to Cassell,None,None,None,None,4.0,406,Shaquille O'Neal,1.610613e+09,Los Angeles,Lakers,LAL,5.0,170,Joe Kleine,1.610613e+09,Phoenix,Suns,PHX,5.0,208,Sam Cassell,1.610613e+09,Phoenix,Suns,PHX,0
2,29600012,3,2,1,1,14:51 PM,11:45,None,None,MISS Cassell 15' Jump Shot,None,None,5.0,208,Sam Cassell,1.610613e+09,Phoenix,Suns,PHX,0.0,0,None,NaN,None,None,None,0.0,0,None,NaN,None,None,None,0
3,29600012,4,4,0,1,14:51 PM,11:43,O'Neal REBOUND (Off:0 Def:1),None,None,None,None,4.0,406,Shaquille O'Neal,1.610613e+09,Los Angeles,Lakers,LAL,0.0,0,None,NaN,None,None,None,0.0,0,None,NaN,None,None,None,0
4,29600012,5,2,1,1,14:51 PM,11:29,MISS Ceballos 26' 3PT Jump Shot,None,None,None,None,4.0,76,Cedric Ceballos,1.610613e+09,Los Angeles,Lakers,LAL,0.0,0,None,NaN,None,None,None,0.0,0,None,NaN,None,None,None,0


In [5]:
# Convert homedescription to lowercase
df['homedescription'] = df['homedescription'].str.lower()

# Filter the DataFrame where homedescription is "lakers rebound"
filtered_df = df[df["homedescription"] == "lakers rebound"]

# Get the value counts for eventmsgtype in the filtered DataFrame
eventmsgtype_counts = filtered_df["eventmsgtype"].value_counts()
eventmsgtype_counts


eventmsgtype
4    9778
Name: count, dtype: int64

In [6]:
df = df.drop(columns=["player2_team_city", "wctimestring", "neutraldescription", "player3_team_city", 
                      "player3_team_abbreviation", "player1_team_city", "player1_team_abbreviation", 
                      "player2_team_abbreviation", "video_available_flag"])

In [7]:
df["player2_team_nickname"].unique()

array([None, 'Suns', 'Lakers', 'Hawks', 'Heat', 'Cavaliers', 'Nets',
       'Clippers', 'Warriors', 'Kings', 'Rockets', 'Bulls', 'Celtics',
       'Knicks', 'Raptors', 'Mavericks', 'Nuggets', 'Bucks', '76ers',
       'Spurs', 'Timberwolves', 'SuperSonics', 'Jazz', 'Bullets', 'Magic',
       'Pacers', 'Pistons', 'Hornets', 'Trail Blazers', 'Grizzlies',
       'Wizards', 'West', 'East', 'Bobcats', 'Thunder', 'Pelicans',
       'Fenerbahce Ulker', 'Basket', 'CSKA', 'Maccabi Haifa',
       'Maccabi Electra', 'Alba Berlin', 'Flamengo',
       'Fenerbahce Sports Club', 'Olimpia Milano', 'Paschoalotto/Bauru',
       'Real Madrid', 'Shanghai Sharks', 'FC Barcelona Lassa',
       'San Lorenzo', 'Long-Lions', 'Sharks', 'United', 'LeBron',
       'Stephen', 'Giannis', 'Durant', "Maccabi Ra'anana", '36ers'],
      dtype=object)

In [8]:
# So after seeing that there are many different team names (some of them are not nba teams or they are not updated). 
# I will change the not to date ones, and drop the games where the non nba teams appear, which are:
non_nba_teams = [
    'East', 'West', 'Fenerbahce Ulker', 'Basket', 'CSKA', 'Maccabi Haifa', 
    'Maccabi Electra', 'Alba Berlin', 'Flamengo', 'Fenerbahce Sports Club', 'Olimpia Milano', 
    'Paschoalotto/Bauru', 'Real Madrid', 'Shanghai Sharks', 'FC Barcelona Lassa', 'San Lorenzo', 
    'Long-Lions', 'Sharks', 'United', 'Stephen', 'LeBron', 'Giannis', 'Durant', 
    "Maccabi Ra'anana", '36ers'
]

# The result will be all the games that I will drop involving non NBA teams or all-star games.
cleaning = df[df['player1_team_nickname'].isin(non_nba_teams)] 
unique_game_ids = cleaning['game_id'].unique()
unique_game_ids

array([39700001, 30300001, 30400001, 30500001, 30600001, 30700001,
       30900001, 31000001, 31100001, 31200001, 11300001, 11300005,
       11300008, 11300009, 11300020, 11300027, 11300046, 31300001,
       11400003, 11400011, 11400022, 11400020, 11400034, 11400055,
       11400054, 11400067, 11400070, 11400075, 11400104, 31400001,
       11500008, 11500016, 11500021, 11500024, 11500030, 11500034,
       11500043, 11500041, 11500069, 31500001, 11600003, 11600010,
       11600020, 11600060, 31600001, 11700005, 11700021, 11700026,
       11700039, 11700055, 31700001, 31800001, 31900001, 32000001,
       32100001, 12200002, 12200008, 12200027, 12200025, 12200038,
       32200001])

In [9]:
game_ids_to_drop = [39700001, 30300001, 30400001, 30500001, 30600001, 30700001,
       30900001, 31000001, 31100001, 31200001, 11300001, 11300005,
       11300008, 11300009, 11300020, 11300027, 11300046, 31300001,
       11400003, 11400011, 11400022, 11400020, 11400034, 11400055,
       11400054, 11400067, 11400070, 11400075, 11400104, 31400001,
       11500008, 11500016, 11500021, 11500024, 11500030, 11500034,
       11500043, 11500041, 11500069, 31500001, 11600003, 11600010,
       11600020, 11600060, 31600001, 11700005, 11700021, 11700026,
       11700039, 11700055, 31700001, 31800001, 31900001, 32000001,
       32100001, 12200002, 12200008, 12200027, 12200025, 12200038,
       32200001]

df = df[~df["game_id"].isin(game_ids_to_drop)]

In [10]:
old_nba_teams = ['SuperSonics', 'Bullets', 'Bobcats']
new_nba_teams = ["Thunder", "Wizards", "Hornets"]

team_name_mapping = dict(zip(old_nba_teams, new_nba_teams))

# Replace the old team names with new team names in both columns
df["player1_team_nickname"] = df["player1_team_nickname"].replace(team_name_mapping)
df["player2_team_nickname"] = df["player2_team_nickname"].replace(team_name_mapping)

df["player1_team_nickname"].unique()
# Okay so dropped all the games where there is a team that is not an NBA teamd AND changed the names of the old teams that are nowadays the equivalent.

array([None, 'Lakers', 'Suns', 'Heat', 'Hawks', 'Nets', 'Cavaliers',
       'Warriors', 'Clippers', 'Rockets', 'Kings', 'Celtics', 'Bulls',
       'Raptors', 'Knicks', 'Nuggets', 'Mavericks', '76ers', 'Bucks',
       'Timberwolves', 'Spurs', 'Jazz', 'Thunder', 'Magic', 'Wizards',
       'Pacers', 'Pistons', 'Hornets', 'Trail Blazers', 'Grizzlies',
       'Pelicans'], dtype=object)

In [13]:
# I will not use the team ID, the name will do
df = df.drop(columns=["player1_team_id", "player2_team_id", "player3_team_id"])